In [ ]:
import torch
import csv
import os
import torchvision.transforms.v2 as transforms_v2
from PIL import Image
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torch import nn
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.cuda.set_device(1)

In [ ]:
train_images = [] 
train_labels = [] 
class_to_idx = {}

transform = transforms_v2.Compose([
                            transforms_v2.ToImage(),
                            transforms_v2.ToDtype(dtype = torch.float32, scale=True),
                            transforms_v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                        ])

flip = transforms_v2.RandomHorizontalFlip(p=0.5)

class_folders = os.listdir(r"C:\Users\sjw00\OneDrive\Desktop\dataset\tiny-imagenet-200\train")
for num, i in enumerate(class_folders):
    class_to_idx[i] = num

for class_folder in class_folders:
    folder_path = os.path.join(r"C:\Users\sjw00\OneDrive\Desktop\dataset\tiny-imagenet-200\train", class_folder) +r"\images"
    temp_image = []
    temp_label = []
    for img_file in os.listdir(folder_path): 
        img_path = os.path.join(folder_path, img_file) 

        img = Image.open(img_path)
        if img.mode != 'RGB':
            img = img.convert('RGB')
        img_tensor = transform(img)  

        train_images.append(img_tensor)
        train_labels.append(torch.tensor(class_to_idx[class_folder],dtype=torch.long))

train_images = torch.stack(train_images)
train_labels = torch.stack(train_labels)
########################################################################
test_images = [] 
test_labels = [] 

with open(r"C:\Users\sjw00\OneDrive\Desktop\dataset\tiny-imagenet-200\val\val_annotations.txt", 'r') as f:
    for line in f:
        file, idx, _ = line.strip().split('\t', 2) # Split each line by space into folder name and index
        img_path = os.path.join(r"C:\Users\sjw00\OneDrive\Desktop\dataset\tiny-imagenet-200\val\images", file) 
        
        img = Image.open(img_path)
        if img.mode != 'RGB':
            img = img.convert('RGB')
        img_tensor = transform(img)  

        test_images.append(img_tensor)
        test_labels.append(torch.tensor(class_to_idx[idx],dtype=torch.long))

test_images = torch.stack(test_images).to("cuda")
test_labels = torch.stack(test_labels).to("cuda")

training_dataset = TensorDataset(train_images, train_labels)
test_dataset = TensorDataset(test_images, test_labels)

training_dataloader = DataLoader(training_dataset, batch_size=500, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=500, shuffle=False)


In [ ]:
#Pytorch 2.5.1
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.layer_1 = nn.Conv2d(3,3,3,stride=1,padding=1)
        self.layer_2 = nn.Conv2d(3,3,3,stride=1,padding=1)
        self.layer_3 = nn.Conv2d(3,3,3,stride=1,padding=1)
        self.layer_4 = nn.Linear(12288,200)
        self.lrelu = nn.LeakyReLU()
        self.flatten = nn.Flatten()
        self.softmax = nn.Softmax(dim=1)
        
        torch.nn.init.zeros_(self.layer_1.weight)
        torch.nn.init.zeros_(self.layer_1.bias)
        torch.nn.init.zeros_(self.layer_2.weight)
        torch.nn.init.zeros_(self.layer_2.bias)
        torch.nn.init.zeros_(self.layer_3.weight)
        torch.nn.init.zeros_(self.layer_3.bias)
        torch.nn.init.zeros_(self.layer_4.weight)
        torch.nn.init.zeros_(self.layer_4.bias)
        
    def forward(self, x):
        x_1 = self.lrelu(self.layer_1(x)) + x
        x_1 = self.lrelu(self.layer_2(x_1)) + x_1
        x_1 = self.lrelu(self.layer_3(x_1)) + x_1
        x_1 = self.flatten(x_1)
        x_1 = self.layer_4(x_1)
        return x_1
    

In [ ]:
def train(dataloader, model, loss_fn, optimizer, scaler, flip):   
    for batch, (X, y) in enumerate(dataloader):
        X = flip(X).to("cuda")
        y = y.to("cuda")
        with torch.autocast(device_type='cuda', dtype=torch.bfloat16):
            pred = model(X)
            batch_loss_result = loss_fn(pred, y)
        optimizer.zero_grad()

        scaler.scale(batch_loss_result).backward() # Backpropagation: compute gradients
        scaler.step(optimizer) # Update model parameters using the optimizer
        scaler.update() # Update the gradient scaler


In [ ]:
def test(dataloader, model):
    with torch.no_grad():
        accuracy_sum=0
        for _, (X, y) in enumerate(dataloader):
            with torch.autocast(device_type='cuda', dtype=torch.bfloat16):
                pred = model(X)
            accuracy_sum+= (torch.argmax(pred, dim=1) == y).type(torch.float).sum().item()

        print("test_accuracy", accuracy_sum/10000)
    return accuracy_sum/10000

In [ ]:
summary = list()
total_list = []
for i in range(10):
    model = CNN().to("cuda")
    loss_fn = nn.CrossEntropyLoss()
    scaler = torch.amp.GradScaler()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.95)
    accuracy = 0 
    epoch_list = []
    for t in range(200):
        print(f"Epoch {t+1}\n-------------------------------")
        train(training_dataloader, model, loss_fn, optimizer, scaler, flip)
        accuracy_temp = test(test_dataloader, model)
        epoch_list.append(accuracy_temp)
        scheduler.step()
        if accuracy_temp > accuracy:
            accuracy = accuracy_temp
    total_list.append(epoch_list)
    summary.append(accuracy)
    print(summary)
with open("tiny_1.csv", mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerows(total_list)
print("Done!")
print(summary)